# **Для использования в Google Colab**

## **Установка**

Установка всех необходимых зависимостей

In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=aa5cf168246ebe450209b29d69f24529a11e3f068c9b7eaf308a9c595d237b2e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


Импортирование библиотек

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, BertConfig

Поскольку глубокое обучение можно значительно ускорить с помощью графического процессора вместо центрального процессора, нужно убедиться, что виртуальная машина использует именно графический процессор. Нужно установить флаг «Время выполнения» — «Изменить тип среды выполнения» — и установить аппаратный ускоритель на «GPU»). Ниже реализовано проверка используемого процессора

In [ ]:
from torch import cuda
device_name = 'cpu'
if cuda.is_available():
  device_name = 'cuda'
  print('Используется графический процессор с ядрами Cuda')
else:
  device_name = 'cpu'
  print('Используется центральный процессор')

Используется графический процессов с ядрами Cuda


После импортирования и настройки среды необходимо загрузить набор данных для обучения модели. Модель распознавания именованных сущностей использует специальную схему для аннотирования слов. Схема аннотирования называется IOB (Inside-Outside-Beginning).

Ниже приведена функция обработки набора данных для приведения к необходимому формату. На вход подается путь до файла, содержащего набор данных.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Загрузка и предварительная обработка наборов данных**

In [ ]:
def create_dataset(file_name):
  with open(file_name, 'r') as dataset:
    file = dataset.readlines()

    unique_labels = []
    sentences = []
    word_labels = []
    sentence_words = []
    sentence_word_labels=[]
    for line in file:
      if line!='\n':
        word, label = line.split('\t')
        stripped_label = label.strip()
        sentence_words.append(word)
        sentence_word_labels.append(stripped_label)
        if stripped_label not in unique_labels:
          unique_labels.append(stripped_label)
      else:
        sentences.append(sentence_words)
        word_labels.append(sentence_word_labels)
        sentence_words = []
        sentence_word_labels=[]
      dataset.close()


  print(len(sentences))
  print(len(word_labels))
  print(unique_labels)

  data_values = pd.DataFrame(columns=['sentence', 'word_labels'])
  for i in range(len(sentences)):
    sentence = " ".join(sentences[i])
    labels = ",".join(word_labels[i])
    data_values.loc[len(data_values.index)] = [sentence, labels] 
  data_values = data_values[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
  data_values.head()

  return data_values, unique_labels



Необходимо инициализировать переменные для конфигурации обучения модели

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Далее необходимо загрузить набор данных, подготовленный в результате работы модуля сбора информации об угрозах безопасности, после чего поместить его в папку облачного хранилища.

Инициализация наборов данных, а также преобразования токенов в уникальные идентификаторы:

In [ ]:
dataset_train, _ = create_dataset('/content/drive/MyDrive/Colab_ML/dataset_train.csv')
dataset_validate, _ = create_dataset('/content/drive/MyDrive/Colab_ML/dataset_valid.csv')
dataset_test, unique_labels = create_dataset('/content/drive/MyDrive/Colab_ML/dataset_test.csv')

label_to_id = {k: v for v, k in enumerate(unique_labels)}
id_to_label = {v: k for v, k in enumerate(unique_labels)}

2810
2810
['B-MW', 'I-MW', 'O', 'B-SYS', 'I-SYS', 'B-ORG', 'B-IOC', 'I-ORG', 'I-IOC', 'B-VULN', 'I-VULN']
812
812
['B-MW', 'O', 'B-IOC', 'I-IOC', 'B-SYS', 'I-SYS', 'B-ORG', 'I-MW', 'I-ORG', 'B-VULN', 'I-VULN']
747
747
['O', 'B-SYS', 'B-ORG', 'I-ORG', 'B-VULN', 'I-VULN', 'B-MW', 'B-IOC', 'I-SYS', 'I-MW', 'I-IOC']


Особенность NER в BERT заключается в том, что BERT использует не просто токенизацию слов, а токенизацию частей слов. Например, слово ringtone будет токенизировано как "ring", "##tone". Для решения данное задачи подходит два способа:
1. Использовать именованную метку только на первой части слова ("ring").
2. Распространять именованную метку на все части слова. Однако для этого необходимы дополнительные преобразования (данный функционал реализован в методе ниже).
3. Использовать необходимую именнованную метку на первой части слова, а на всех остальных другую именованную метку.

In [ ]:
def tokenize_fragments(sentence, text_labels, tokenizer):
    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Токенизация слова и подсчет количества подслов
        tokenized_word = tokenizer.tokenize(word)
        subwords_len = len(tokenized_word)

        # Добавление токенизированного слова в массив токенизированного предложения
        tokenized_sentence.extend(tokenized_word)

        # Продублировать именованную метку на все подслова
        labels.extend([label] * subwords_len)

    return tokenized_sentence, labels

В классе, описанном ниже происходит преобразование кадров данных в тензоры PyTorch. Каждое предложение токенизируется специальными токенами, которые принимает на вход BERT. Сами токены либо дополняются, либо усекаются, в зависимости от длины, указанной в параметрах выше.

In [ ]:
class CustomDataset(Dataset):
    len = None
    max_len = None
    data = None
    tokenizer = None
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.max_len = max_len
        self.data = dataframe
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):
        # токенизация предложения (включая подслова)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_fragments(sentence, word_labels, self.tokenizer)
        
        # добавление специальных токенов и меток
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # добавление специальных токенов BERT (классификатор и сепаратор)
        labels.insert(0, "O") # добавление метки для [CLS] токена
        labels.insert(-1, "O") # добавление метки для [SEP] токена

        # дополнение/усечение
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # усечение
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # дополнение соответствующим токеном BERT
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # получение "маски внимания"
        attention_mask = [1 if token != '[PAD]' else 0 for token in tokenized_sentence]
        
        # конвертация токенов и токенизированных предложений в идентификаторы
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label_to_id[label] for label in labels]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attention_mask, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

С помощью созданного выше класса необходимо преобразовать наборы данных в наборы для обучения модели BERT

In [ ]:
print("Обучающий набор данных: {}".format(dataset_train.shape))
print("Тестовый набор данных: {}".format(dataset_test.shape))
print("Валидационный набор данных: {}".format(dataset_validate.shape))

training_set = CustomDataset(dataset_train, tokenizer, MAX_LEN)
testing_set = CustomDataset(dataset_test, tokenizer, MAX_LEN)
validating_set = CustomDataset(dataset_validate, tokenizer, MAX_LEN)

Обучающий набор данных: (2746, 2)
Тестовый набор данных: (746, 2)
Валидационный набор данных: (809, 2)


Далее необходимо инициализировать соответствующие загрузчики данных PyTorch.

In [ ]:
training_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

validating_params = {'batch_size': VALID_BATCH_SIZE,
              'shuffle': True,
              'num_workers': 0
              }

testing_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
              


training_loader = DataLoader(training_set, **training_params)
validating_loader = DataLoader(validating_set, **validating_params)
testing_loader = DataLoader(testing_set, **testing_params)

## **Создание, обучение, проверка и тестирование модели**

Теперь необходимо инициализировать модель "BertForTokenClassification", задав в параметрах инициализацию базовых слоев с весами, полученными в результате предварительного обучения "bert-base-uncased". Оставльные слои имеют случайные веса, которые будут меняться в процессе обучения на созданном наборе данных.

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id_to_label),
                                                   id2label=id_to_label,
                                                   label2id=label_to_id)
model.to(device_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

Далее необходимо инициализировать оптимизатор. Он позволяет менять атрибуты нейронной сети, такие как веса или же скорость обучения. Таким образом, это помогает уменьшить общие потери и повысить точность.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

После необходимо определить тренировочную функцию.

In [ ]:
# Определение функции обучения на 80% набора данных для настройки модели BERT
def train(epoch):
    training_loss, training_accuracy = 0, 0
    nb_training_examples, nb_training_steps = 0, 0
    training_predictions, training_labels = [], []
    # Перевод модели в режим обучения
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device_name, dtype = torch.long)
        mask = batch['mask'].to(device_name, dtype = torch.long)
        targets = batch['targets'].to(device_name, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        training_loss += loss.item()

        nb_training_steps += 1
        nb_training_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = training_loss/nb_training_steps
            print(f"Потеря на 100 шагов обучения: {loss_step}")
           
        # Подсчет точности обучения
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # Использование маски, чтобы определить, где нужно сравнивать прогнозы с целями (включая прогнозы токенов [CLS] и [SEP])
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        training_predictions.extend(predictions)
        training_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        training_accuracy += tmp_tr_accuracy
    
        # Градиентная нормализация
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # Обратный проход
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = training_loss / nb_training_steps
    training_accuracy = training_accuracy / nb_training_steps
    print(f"Потеря обучения эпохи: {epoch_loss}")
    print(f"Точность обучения эпохи: {training_accuracy}")

Непосредственное обучение модели.

In [ ]:
for epoch in range(EPOCHS):
    print(f"Обучение эпохи: {epoch + 1}")
    train(epoch)

Обучение эпохи: 1
Потеря на 100 шагов обучения: 2.4730238914489746
Потеря на 100 шагов обучения: 0.4696225239203708
Потеря на 100 шагов обучения: 0.31188817588677303
Потеря на 100 шагов обучения: 0.24162739783872006
Потеря на 100 шагов обучения: 0.20376178980026943
Потеря на 100 шагов обучения: 0.17709810475627344
Потеря на 100 шагов обучения: 0.15992531259451218
Потеря обучения эпохи: 0.14722437155331328
Точность обучения эпохи: 0.9004559684064236
Обучение эпохи: 2
Потеря на 100 шагов обучения: 0.04690247401595116
Потеря на 100 шагов обучения: 0.05631408438151057
Потеря на 100 шагов обучения: 0.05352568165433885
Потеря на 100 шагов обучения: 0.05064806044473007
Потеря на 100 шагов обучения: 0.05051117640672637
Потеря на 100 шагов обучения: 0.0493402048950539
Потеря на 100 шагов обучения: 0.048233423642961376
Потеря обучения эпохи: 0.04743891164797373
Точность обучения эпохи: 0.9535781899316477
Обучение эпохи: 3
Потеря на 100 шагов обучения: 0.007390196435153484
Потеря на 100 шагов обу

После обучения модели можно оценить ее производительность на валидационном наборе данных (20% от общего количества данных). Следует обратить внимание, что модель не обновляет градиенты.

In [ ]:
def valid(model, val_loader):
    # Перевод модели в режим проверки
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(val_loader):
            
            ids = batch['ids'].to(device_name, dtype = torch.long)
            mask = batch['mask'].to(device_name, dtype = torch.long)
            targets = batch['targets'].to(device_name, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Потеря проверки на 100 шагов: {loss_step}")
              
            # Подсчет точности проверки
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # Использование маски, чтобы определить, где нужно сравнивать прогнозы с целями (включая прогнозы токенов [CLS] и [SEP])
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    
    labels = [id_to_label[id.item()] for id in eval_labels]
    predictions = [id_to_label[id.item()] for id in eval_preds]

       
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Потеря проверки: {eval_loss}")
    print(f"Точность проверки: {eval_accuracy}")

    return labels, predictions

Запуск проверки обучения модели.

In [ ]:
labels, predictions = valid(model, validating_loader)

Потеря проверки на 100 шагов: 0.029252657666802406
Потеря проверки на 100 шагов: 0.04827219610636141
Потеря проверки на 100 шагов: 0.04618635125758256
Потеря проверки на 100 шагов: 0.04502811763782528
Потеря проверки на 100 шагов: 0.04139922827401226
Потеря проверки: 0.04152996955607604
Точность проверки: 0.9547556147344403


Аналогично двум предыдущим функциям, нужно добавить функцию тестирования модели для оценки точности.

In [ ]:
def testing(model, testing_loader):
    # Применить режим проверки для модели
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device_name, dtype = torch.long)
            mask = batch['mask'].to(device_name, dtype = torch.long)
            targets = batch['targets'].to(device_name, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Потеря тестирования на 100 шагов: {loss_step}")
              
            # Подсчет точности тестирования
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # Использование маски, чтобы определить, где нужно сравнивать прогнозы с целями (включая прогнозы токенов [CLS] и [SEP])
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    

    labels = [id_to_label[id.item()] for id in eval_labels]
    predictions = [id_to_label[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Потеря тестирования: {eval_loss}")
    print(f"Tочность тестирования: {eval_accuracy}")

    return labels, predictions

Запуск тестирования модели

In [ ]:
labels, predictions = testing(model, testing_loader)

Потеря тестирования на 100 шагов: 0.005713593680411577
Потеря тестирования на 100 шагов: 0.05297955934656784
Потеря тестирования на 100 шагов: 0.06086458839619166
Потеря тестирования на 100 шагов: 0.05839774745295638
Потеря тестирования: 0.05880815476998712
Tочность тестирования: 0.9502036348163433


Однако метрика точности вводит в заблуждение, поскольку многие метки – это метки «вне» (O). Что важно, так это смотреть на точность, полноту и F1-меру отдельных тегов. Для этого используется библиотека seqeval.

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         IOC       0.82      0.85      0.83      2261
          MW       0.72      0.74      0.73       603
         ORG       0.62      0.32      0.42       241
         SYS       0.40      0.40      0.40       399
        VULN       1.00      0.17      0.29        47

   micro avg       0.75      0.74      0.74      3551
   macro avg       0.71      0.50      0.54      3551
weighted avg       0.75      0.74      0.73      3551



После того, как модель обучена и показывает достаточные для задачи результаты, можно попытаться выделить сущности в новом предложении.

## **Пример применения обученной модели на реальных данных**

In [ ]:
sentence = "Technical analysis Most of this new attack ’ s routines are similar to those of the previous XLoader versions"

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# Вычисление на графическом процессоре
ids = inputs["input_ids"].to(device_name)
mask = inputs["attention_mask"].to(device_name)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - Предсказания на уровне токенов

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id_to_label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # Список кортежей. Каждый кортеж = (подслово, предсказание)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # Не предсказывать подслова и специальные токены
    continue
  else:
    word_level_predictions.append(pair[1])

# Восстановление предложения без специальных токенов
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

technical analysis most of this new attack ’ s routines are similar to those of the previous xloader versions
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MW', 'B-MW', 'B-MW', 'O']


## **Сохранение модели**

После всех проверок следует сохранить файлв словаря, веса и конфигурация модели для возможности повторной загрузки с помощью метода "from_pretrained()" для предобученных моделей.

In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# Сохранение словаря токенизатора
tokenizer.save_vocabulary(directory)
# Сохранение конфигурационного файла и весов модели
model.save_pretrained(directory)
print('Все файлы сохранены')

Все файлы сохранены
